In [32]:
import numpy as np
from scikits.odes.ode import ode
import matplotlib.pyplot as plt
import random
from IPython.utils import io



In [2]:
#Import local functions
sys.path.append('../functions')
from functions import nonlinearity, repression, activation

In [3]:
#Default information - user can change but mostly won't
#Constants (set to default)
kc=12.; km=10.; lam=1.93E-4; Vin=1.; e0=0.0467

#Integration conditions
y0 = np.array([2290., 0., 0., 0., 0., 0.]) 
times = np.linspace(0, 5e4, 100) 

#User specified information
X = 2 #Number of chemical species in pathway
E = 2 #Number of enzymes in pathway

#S = np.zeros([X, E])
S = np.array([[ -1, 0], [1, -1]]) #Stoichiometric matrix defining chemical reactions

I = np.zeros(X) #Input matrix defining inputs to pathway
I[0] = Vin #Inputs to pathway

T = [1] #Xs that can act as TFs

In [4]:
params = np.zeros([len(T), E, 3])
#Manually set initial conditions
for t in range(len(T)):
    for e in range(E):
        params[t][e] = [2., 0.1,1E-4]

In [5]:
def generate_equation(A, W):
    '''Generate genetic equations from architecture matrix'''
    def generated_equation(t, y, ydot):
        '''Generate function to solve with Scikit-ODEs'''
        ydot[0] = Vin - lam*y[0] - e0*nonlinearity(y[0], kc, km) - y[2]*nonlinearity(y[0], kc, km)
        ydot[1] = y[2]*nonlinearity(y[0], kc, km) - y[3]*nonlinearity(y[1], kc, km) - lam*y[1]
        for e in range(E):
            ydot[e+X] = -lam*y[e+X] + np.sum([np.sum(A[t][e]*[activation(y[T[t]], W[t][e][2], W[t][e][1], W[t][e][0]), repression(y[T[t]], W[t][e][2], W[t][e][1], W[t][e][0]), W[t][e][2]]) for t in range(len(T))])
        ydot[E+X] = (I[0] - y[X+1]*nonlinearity(y[X-1], kc, km))**2 #J1
        ydot[E+X+1] = np.sum([np.sum([np.sum(A[t][e]*[activation(y[T[t]], W[t][e][2], W[t][e][1], W[t][e][0]), repression(y[T[t]], W[t][e][2], W[t][e][1], W[t][e][0]), 0]) for t in range(len(T))]) for e in range(E)]) #J2
    return generated_equation

In [6]:
def loss_biological(j1, j2, alpha1, alpha2):
        """Computes scalarized loss including genetic constraint and product production"""
        loss = alpha1*j1 + alpha2*j2
        return j1, j2, loss

In [7]:
def bounds_check(params):
    n1, n2 = params[0][:, 0]
    theta1, theta2 = params[0][:, 1]
    k1, k2 = params[0][:, 2]
    if k1 <= 1E-3 and k2 <= 1E-3 and k1 >= 1E-7 and k2 >= 1E-7:
        if n1 <= 4 and n2 <= 4 and n1 >= 1 and n2 >= 1:
            if theta1 >= 0.001 and theta2 >= 0.001 and theta1 <= 10 and theta2 <= 10:
                return True
    else: return False

In [8]:
def generate_initial_params(N):
    #Set random seed
    np.random.seed(2022)
    ics = []
    for i in range(N):
        k1 = random.uniform(1E-7,1E-3)
        k2 = random.uniform(1E-7,1E-3)
        n1 = random.uniform(1,4)
        n2 = random.uniform(1,4)
        theta1 = random.uniform(0.001,10)
        theta2 = random.uniform(0.001,10)

        init_conds = np.array([[[n1, theta1, k1], [n2, theta2, k2]]])
        ics.append(init_conds)
    return ics

In [36]:
def solve_single(A, W, times, y0):
    ode_function = generate_equation(A, W)
    with io.capture_output() as captured:
        solution = ode('cvode', ode_function, old_api=False).solve(times, y0)
    j1, j2 = solution.values.y[-1, -2:]
    j1, j2, loss = loss_biological(j1, j2, alpha1=1E-5, alpha2= 1E-2)
    return loss

In [10]:
def solve_patch(A, params, times, y0, patch_size, step_sizes):
    center_loss = solve_single(A, params, times, y0)
    #Sample randomly from hypersphere
    normal_deviates = np.random.normal(size=(patch_size, params.shape[0], params.shape[1], params.shape[2]))
    radius = np.sqrt((normal_deviates**2).sum(axis=0))
    points = normal_deviates/radius
    scaled_points = step_sizes*points + params
    
    min_loss = center_loss
    new_params = params
    for scaled_params in scaled_points:         
        loss = solve_single(A, scaled_params, times, y0)
        if bounds_check(scaled_params):
            if loss < min_loss:
                new_params = scaled_params
                min_loss = loss
    return new_params, min_loss

In [21]:
def solve_architecture(A, times, y0, num_initializations=1, num_epochs = 100, tolerance=0.0004, patch_size=100, step_sizes= np.array([[[0.1, 0.01, 0.001], [0.1, 0.01, 0.0001]]])):
    initial_params = generate_initial_params(num_initializations)
    for i in range(len(initial_params)):
        print('Descent', i+1, 'of', len(initial_params))
        init_conds = initial_params[i]
        next_params, loss = solve_patch(A, init_conds, times, y0, patch_size, step_sizes)
        losses = [loss]; param_trace = [next_params]

        for i in range(num_epochs):
            new_params, loss = solve_patch(A, next_params,times, y0,patch_size, step_sizes)
            next_params = new_params

            print('Epoch: ', i, 'Loss: ', loss)
            losses.append(loss)
            param_trace.append(new_params)
            
            #Early stopping criterion
            if losses[-2] - losses[-1] < tolerance:
                print('Terminating early at step', i, 'of ', num_epochs)
                break
    return param_trace, losses



In [29]:
A = np.zeros([len(T), E, 3]) #3 methods of action at each locus for each TF
A[0][0] = [0, 1, 0]
A[0][1] = [0, 0, 1]

In [37]:
param_trace, losses = solve_architecture(A, times, y0, num_initializations=1, num_epochs = 100, tolerance=0.0004, patch_size=100, step_sizes= np.array([[[0.1, 0.01, 0.001], [0.1, 0.01, 0.0001]]]))

Descent 1 of 1
Epoch:  0 Loss:  0.1008471560076038



[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.



Epoch:  1 Loss:  0.04469488311968336



[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13

Epoch:  2 Loss:  0.04465044559737107
Terminating early at step 2 of  100



[CVODE ERROR]  CVode
  At t = 3.34879e-12 and h = 1.27746e-13, the corrector convergence test failed repeatedly or with |h| = hmin.

